
# CNN-based Multi-Class Flower Classification

### Import Necessary Libraries

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Load and Preprocess Data

**Data Preprocessing Overview**
Here, we'll load the flower dataset and preprocess it. The steps include:
- Loading the image dataset from the given directory paths.
- Normalizing pixel values to the range [0, 1].
- Splitting the data into training, validation, and testing sets.

In [ ]:
# Load and Preprocess Data

# Define paths to the dataset
train_dir = 'path/to/train'
validation_dir = 'path/to/validation'

# Create ImageDataGenerator instances for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Load and preprocess the validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

### Build CNN Model

**CNN Architecture Details**
We'll build a convolutional neural network (CNN) with the following components:
- **Conv2D Layers**: Extract spatial features using filters.
- **MaxPooling Layers**: Reduce spatial dimensions while retaining key information.
- **Dropout Layers**: Prevent overfitting by randomly dropping neurons.
- **Dense Layers**: Perform classification based on extracted features.

In [ ]:
# Build the CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

# Summarize the model
model.summary()

### Compile the Model

**Compilation Details**
The model is compiled with:
- Loss Function: `categorical_crossentropy`, suitable for multi-class classification.
- Optimizer: `adam`, an efficient gradient descent optimizer.
- Metrics: `accuracy`, to monitor classification performance.

In [ ]:
# Compile the CNN model with appropriate loss function, optimizer, and metrics
model.compile(
    loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
    optimizer=tf.keras.optimizers.Adam(),  # Use Adam optimizer
    metrics=['accuracy']  # Track accuracy as the performance metric
)

### Train the Model

**Training Overview**
We'll train the model using the training dataset and validate it using the validation dataset. The number of epochs and batch size are adjustable hyperparameters.

In [ ]:
# Train the CNN model using the training data and validate it using the validation data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the trained model
model.save('flower_classification_model.h5')

# Plot the training and validation accuracy and loss
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Evaluate the Model

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('flower_classification_model.h5')

# Evaluate the model on the validation data
loss, accuracy = model.evaluate(validation_generator)

print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

# Generate classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get the true labels and predicted labels
validation_generator.reset()
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes

# Print classification report
print('Classification Report')
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

# Print confusion matrix
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

### Make Predictions

In [ ]:
# Make Predictions

# Load the saved model
model = tf.keras.models.load_model('flower_classification_model.h5')

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Function to make predictions on a single image
def make_prediction(image_path):
    img_array = load_and_preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    class_indices = {v: k for k, v in validation_generator.class_indices.items()}
    predicted_label = class_indices[predicted_class[0]]
    return predicted_label

# Example usage
image_path = 'path/to/new/flower/image.jpg'
predicted_label = make_prediction(image_path)
print(f'The predicted class for the image is: {predicted_label}')